#PySpark Practice Notebook

In [5]:
#Source Data
#Create a csv file for the following data
%%writefile country.csv
product_id,product,country,sales
1,Product A,USA,100
2,Product B,USA,80
3,Product C,USA,30
1,Product A,Canada,60
2,Product B,Canada,40
4,Product D,UK,50
5,Product E,UK,20
1,Product A,Germany,70
3,Product C,Germany,90
4,Product D,Germany,40

Writing country.csv


In [1]:

pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488490 sha256=0739f2fdc8b8f1f734d6b13b5e7ad6a2c0bbe1a6a02b3d9ea20a6084b37852dc
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [14]:
#Create SparkSession for app "Sales Data Analysis"
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [3]:
#Create a pyspark DataFrame from the csv file on local storage
arr=SparkSession.builder.appName('nikhil1').getOrCreate()

In [18]:
df=arr.read.option('Header','True').csv('country.csv')

In [22]:
#Display all entries for country "Germany"
# df.filter(df['country']=='Germany').show()
df.filter(df.country == 'Germany').show()

+----------+---------+-------+-----+
|product_id|  product|country|sales|
+----------+---------+-------+-----+
|         1|Product A|Germany|   70|
|         3|Product C|Germany|   90|
|         4|Product D|Germany|   40|
+----------+---------+-------+-----+



In [ ]:
#Find the total number of products
df.select('product_id').count()

10

In [23]:
#Find top 3 products
#df.sort(df['sales'],ascending=False).show(3)
df.orderBy('sales',ascending=False).limit(3).show()


+----------+---------+-------+-----+
|product_id|  product|country|sales|
+----------+---------+-------+-----+
|         3|Product C|Germany|   90|
|         2|Product B|    USA|   80|
|         1|Product A|Germany|   70|
+----------+---------+-------+-----+



In [28]:
from pyspark.sql.functions import col,sum,min,max,avg


In [24]:
#Calculate total sales
#df.select(sum('sales')).show()
df.agg({'sales':'sum'}).show()

+----------+
|sum(sales)|
+----------+
|     580.0|
+----------+



In [ ]:
df.select(min('sales')).show()

+----------+
|min(sales)|
+----------+
|       100|
+----------+



In [ ]:
df.select(max('sales')).show()

+----------+
|max(sales)|
+----------+
|        90|
+----------+



In [ ]:
df.select(('sales')).show()

+----------+
|avg(sales)|
+----------+
|      58.0|
+----------+



In [ ]:
# prompt: #Find the market share of all products individually

# Calculate total sales for each product
product_sales = df.groupBy('product').agg(sum('sales').alias('total_sales'))

# Calculate total sales across all products
total_sales = product_sales.agg(sum('total_sales')).collect()[0][0]

# Calculate market share for each product
product_market_share = product_sales.withColumn('market_share', (col('total_sales') / total_sales) * 100)

# Display the market share of all products individually
product_market_share.show()


In [ ]:
import pyspark.sql.functions as func

In [52]:
#Find the market share of all products individually
x=df.agg({'sales':'sum'}).collect()[0][0]
df.withColumn('market_share',col('sales')/x*100).show()





+----------+---------+-------+-----+------------------+
|product_id|  product|country|sales|      market_share|
+----------+---------+-------+-----+------------------+
|         1|Product A|    USA|  100| 17.24137931034483|
|         2|Product B|    USA|   80|13.793103448275861|
|         3|Product C|    USA|   30| 5.172413793103448|
|         1|Product A| Canada|   60|10.344827586206897|
|         2|Product B| Canada|   40| 6.896551724137931|
|         4|Product D|     UK|   50| 8.620689655172415|
|         5|Product E|     UK|   20|3.4482758620689653|
|         1|Product A|Germany|   70|12.068965517241379|
|         3|Product C|Germany|   90|15.517241379310345|
|         4|Product D|Germany|   40| 6.896551724137931|
+----------+---------+-------+-----+------------------+

